In [1]:
import os
import pandas as pd
import numpy as np

from inro.emme.matrix import MatrixData
import inro.modeller as m
mm = m.Modeller()
eb = mm.emmebank

# toolbox tools
import_csv = "inro.emme.data.matrix.import_matrix_from_csv"
import_csv_mat = mm.tool(import_csv)

tmg_import_binary = "tmg.input_output.import_binary_matrix"
tmg_import = mm.tool(tmg_import_binary)

tmg_export_binary = "tmg.input_output.export_binary_matrix"
tmg_export = mm.tool(tmg_export_binary)

def mat_to_pddf(id, scen):
    mat = eb.matrix(id).get_data(scen)
    zones_tupl = mat.indices
    indices = pd.Index(zones_tupl[0])
    indices.name = 'p'
    cols = pd.Index(zones_tupl[1])
    cols.name = 'q'
    matrix = mat.to_numpy()
    return pd.DataFrame(matrix, index=indices, columns=cols)

def pddf_to_mat(dataframe, mtx_out=None, scenario_id=None):
    indices = list(dataframe.index.values), list(dataframe.columns.values)
    md = MatrixData(indices)
    array = dataframe.values
    if array.flags.f_contiguous:
        array = np.ascontiguousarray(array)
    md.from_numpy(array)

ModuleNotFoundError: No module named 'inro'

In [2]:
### Inputs
file_path = r"C:\Users\pechen\Desktop\407ETR\407 Update\407_Update_EMME\V4Input\Demand_Adjustment\2016"
files = ["AM - HighDA - 20220113.mtx", "PM - HighDA - 20220113.mtx"]

## Outputs
out_file_path = r"C:\Users\pechen\Desktop\407ETR\407 Update\407_Update_EMME\V4Input\Demand_Adjustment\2016"
out_files = ["AM - HighDA - 3,0.25cap - 20220216.mtx", "PM - HighDA - 3,0.25cap - 20220216.mtx"]


In [3]:
scen_num = 11 #Scenario number - must exist
temp_matrix = "mf200" #Temporary matrix - must exist

scen = eb.scenario(scen_num) # scenario
mat = eb.matrix(temp_matrix) # temp matrix

for i in range(len(files)):
    tmg_import(4, temp_matrix[2:], os.path.join(file_path, files[i]), scen_num, "temp")
    
    mat_df = mat_to_pddf(temp_matrix, scen_num)
    
    ### ADJUSTMENTS
    mat_df[mat_df < 0.25] = 0.25
    mat_df[mat_df > 3] = 3
    ###

    mat_df.to_csv(os.path.join(out_file_path, out_files[i][:-4] +'.csv'))
    import_csv_mat(mat, os.path.join(out_file_path, out_files[i][:-4] +'.csv'), scen)
    tmg_export(4, temp_matrix[2:], os.path.join(out_file_path, out_files[i]), scen_num)